Erster Teil von https://python.langchain.com/docs/use_cases/tool_use/agents/ 

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

In [ ]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

In [ ]:
# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)

### Jetzt bauen wir mal den Javaland-Vectorstore ein:

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

db2 = Chroma(persist_directory="./chroma", embedding_function=OpenAIEmbeddings())

@tool
def search_javaland(query: str) -> str:
    "Searchs for sessions and talks in the Javaland conference agenda, returns descriptions of talks that match the query"
    docs = db2.similarity_search(query)
    contents = map(lambda doc: doc.page_content, docs)
    return "/n".join(contents)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, [search_javaland], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[search_javaland], verbose=True)

result = agent_executor.invoke(
    {
        "input": "Are there cool AI or ChatGPT talks at the Javaland?"
    }
)

In [ ]:
from pprint import pprint

pprint(result["output"])